In [ ]:
import pandas as pd
import numpy as np
import json
import datetime as dt
from datetime import datetime, date, time,timedelta
import pandas as pd
from csv import DictWriter
import urllib
import re,datetime
import os
import time
from os import walk
import re
import  gc
from pyxirr import xirr
import math



from scipy.stats import norm

from datetime import datetime, date, time,timedelta

from tqdm.notebook import tqdm
trigger = [100,25,50,75,125,150, 175, 200]

list1=[100,25,50,125,150,175,200]
N1=[2]#,1,2]#3,1,2]#,3,4,0]
import warnings
warnings.filterwarnings("ignore")
leverage=6
def closest_strike(group, input_value):
    return     (np.abs(group['OPT_Close'] - input_value)).argmin()
def ce_blackscholes(Spot_bn,Strike_bn,YTE,iv_bn_ce):
    NormDist = norm.cdf
    Rate = 0.065
    Dividend = 0.015
    CallPremium=0
    PutPremium=0
    dOne=0
    dTwo=0
    ndOne=0
    ndTwo=0
    dOne = (math.log(Spot_bn/Strike_bn)+((Rate - Dividend + (0.5*((iv_bn_ce)**2)))*YTE))/((iv_bn_ce)*(YTE**0.5))
    dTwo = dOne - (iv_bn_ce) * YTE** 0.5
    ndTwo = NormDist(dTwo)
    ndOne = NormDist(dOne)

    CallPremium =  (math.exp(-Dividend * YTE) * (Spot_bn * ndOne)) - (Strike_bn * math.exp(-Rate * YTE) * ndTwo) 
    return CallPremium
    
def pe_blackscholes(Spot_bn,Strike_bn,YTE,iv_bn_pe):
    NormDist = norm.cdf
    Rate = 0.065
    Dividend = 0.015
    dOne = (math.log(Spot_bn/Strike_bn)+((Rate - Dividend + (0.5*((iv_bn_pe)**2)))*YTE))/((iv_bn_pe)*(YTE**0.5))
    dTwo = dOne - (iv_bn_pe) *YTE ** 0.5
    ndTwo = NormDist(dTwo)
    ndOne = NormDist(dOne)

    PutPremium1 =  math.exp(-Rate * YTE) * Strike_bn * (NormDist(-dTwo)) -math.exp(-Dividend *YTE) *  Spot_bn* (NormDist( -dOne))
    return PutPremium1
    
    


delta_list1=['50']
BN_data=pd.read_csv(r'C:\Projects\Straddles\Data\Monthly/BN-1 test.csv',dayfirst=True,parse_dates=['Date'])
BN_data['Date']=pd.to_datetime(BN_data['Date'])
direc=r'C:\Projects\Data\BN\2023\Monthly/'
monthly_exp=pd.read_csv(r'C:\Projects\Straddles\NEW/MonthlyExpiry_2023.csv',parse_dates = ["curr_exp_date","Date"],dayfirst =True,usecols = ["curr_exp_date","Date"])
delta_list=['50','45', '40', '35', '30', '25','20','15']

for delta in tqdm(delta_list):
    print(delta)
    idx = pd.read_csv(direc + "/BANKNIFTY-I.csv",usecols = ["Date","Ticker","Option_Type","Strike",
                                                                        "OPT_Close","EQ_Close","IV","Delta_"+delta+"_Strike"],parse_dates = ["Date"],dayfirst=True)
    og_idx=pd.read_csv(r'C:\Projects\Data\BN\2023\Monthly\without contract filter' + "/BANKNIFTY-I.csv",usecols = ["Date","Ticker","Option_Type","Strike",
                                                                        "OPT_Close","EQ_Close","IV","Delta_"+delta+"_Strike"],parse_dates = ["Date"],dayfirst=True)
    og_idx['Date']=pd.to_datetime(og_idx['Date'])
    idx=idx[idx['Date']>=pd.to_datetime('27-05-2016')].reset_index(drop=True)
    idx=idx[idx['Date']<=pd.to_datetime('25-01-2023')].reset_index(drop=True)
    og_idx=og_idx[og_idx['Date']>=pd.to_datetime('27-05-2016')].reset_index(drop=True)
    for nnn in tqdm(N1):
            for t in trigger:
                nnn1=nnn
                output_path=r'C:\Projects\Straddles\NEW\2016-2023/Monthly adj/'+delta+'_delta'+'/'+str(nnn)+'/'+str(t)

                print(output_path)
                # print(output_path)
                if not os.path.exists(output_path):
                    os.makedirs(output_path)



                idx_ce = idx[(idx["Strike"] == idx["Delta_"+delta+"_Strike"]) & (idx["Option_Type"] == "CE")].add_suffix("_CE")
                idx_pe = idx[(idx["Strike"] == idx["Delta_"+delta+"_Strike"]) & (idx["Option_Type"] == "PE")].add_suffix("_PE")
                # idx_ce3 = idx3[(idx3["Strike"] == idx3["Delta_"+delta+"_Strike"]) & (idx3["Option_Type"] == "CE")].add_suffix("_w3")

                # idx_pe = idx[(idx["Strike"] == idx["Delta_"+delta+"_Strike"]) & (idx["Option_Type"] == "PE")].add_suffix("_PE")
                final=idx_ce.merge(idx_pe,left_on='Date_CE',right_on='Date_PE')
                final = pd.merge(final, monthly_exp, left_on = 'Date_CE', right_on='Date',how = 'left')
                # final=final.merge(idx_ce3,left_on='Date_short',right_on='Date_w3')

                # final = idx_ce.merge(idx_pe, left_on='Date_CE', right_on='Date_PE').drop(columns = ["Option_Type_PE","Option_Type_CE"])
                if final['Strike_PE'].equals(final['Strike_CE']):
                    print(1,delta,t)
                final = final.T.drop_duplicates().T  #Drop Duplicates Columns
                if 'Strike_PE' not in final.columns:
                    final['Strike_PE']=final['Strike_CE']
                    print(delta,t)
                final.rename(columns = {'Date_CE':'Date'}, inplace = True)

                final.to_csv(output_path+"/check-1data-w3.csv")


                final=pd.read_csv(output_path+"/check-1data-w3.csv",parse_dates=['Date','curr_exp_date'],dayfirst=True)

                df=pd.read_csv(output_path+"/check-1data-w3.csv")
                status=1
                if nnn>0:
                    nnn1=nnn
                    nnn1=nnn1-1
                # print(2,nnn1,nnn)
                for idi,row in df.iterrows():
                # df.loc[idi,'status']=status
                    if nnn!=4:
                        if idi==0:
                            if nnn!=0:
                                df.loc[idi:idi+nnn1-1,'Status']=0
                                df.loc[idi+nnn1,"Marker"]='F'
                                df.loc[idi+nnn1,'status']=1
                            else:
                                df.loc[idi,"Marker"]='F'
                                df.loc[idi,'status']=1
                        elif (df.loc[idi-1,'Date'] == df.loc[idi-1,'curr_exp_date']):
                            # df.loc[idi,"Marker"]='M'
                            if nnn!=0:
                                # print(idi)
                                df.loc[idi:idi+nnn1-1,"Status"]=0
                            df.loc[idi+nnn1,"Marker"]='F'
                            df.loc[idi+nnn1,"status"]=1
                            # df.loc[idi+nnn-1,"status"]=1
                            df.loc[idi+nnn1,"Marker2"]='F'

                        elif (df.loc[idi,'Date'] == df.loc[idi,'curr_exp_date']):
                            df.loc[idi,"Marker"]='L'
                            df.loc[idi,'status']=1
                    elif nnn==4:
                        if idi==0:
                                df.loc[idi+nnn1,"Marker"]='F'
                                df.loc[idi+nnn1,'status']=1
                        elif (df.loc[idi,'Date'] == df.loc[idi,'curr_exp_date']):
                            df.loc[idi-1,"Marker"]='F'
                            df.loc[idi,"Marker"]='L'
                            df.loc[idi-1:idi,'status']=1
                        if df.loc[idi,'status']!=1:
                            df.loc[idi,'Status']=0

                df.reset_index(drop=True,inplace=True)    
                df.to_csv(output_path+'/dte_test3-3-3-1.csv')
                
                # print(3,nnn1)
                for idi,row in df.iterrows():
                    if nnn1!=0 and nnn!=4:
                        if df.loc[idi,'Status']==0:
                            df.drop(idi, axis=0, inplace=True)
                    if nnn==4:
                        if df.loc[idi,'status']!=1:

                            df.drop(idi, axis=0, inplace=True)

                df.to_csv(output_path+'/dte_test3-3-3.csv')
                # print(4,nnn1)
                from tqdm.notebook import tqdm
                final=pd.DataFrame()
                dfg=df.groupby(['curr_exp_date'])
                temp=0
                # print(1,output_path)
                for name,group in tqdm(dfg):
                    
                    if len(group)==1:
                        continue

                    if temp==0:
                        strike_ce=group["Strike_CE"].iat[0]
                        ce_optclose=group["OPT_Close_CE"].iat[0]
                        date=group["Date"].iat[0]
                        strike_pe=group["Strike_PE"].iat[0]

                    if temp==1:
                        strike_ce=strike_new_ce
                        ce_optclose=ce_close_new#group["OPT_Close_short"].iat[0]
                        date=group["Date"].iat[0]
                        strike_pe=strike_new_pe
                        pe_optclose=pe_close_new

                    strike_ce_w2=strike_ce
                    strike_pe_w2=strike_pe


                    x1=x2=x3=y1=y2=y3=0
                    temp1=0
                    group['Date'] = pd.to_datetime(group['Date'], dayfirst=True)
                    group['curr_exp_date'] = pd.to_datetime(group['curr_exp_date'], dayfirst=True)
                    group['YTE'] = (group['curr_exp_date'] - group['Date']).dt.days
                    group['YTE/365'] = group['YTE']/365
                    for idi,row in group.iterrows():
                            if row['Marker']=='F':
                        # if temp==0:
                                date=group.loc[idi,'Date']
                                x_ce=idx[(idx['Date']==date) & (idx["Option_Type"] == "CE") & (idx["Strike"]==strike_ce)]
                                x_pe=idx[(idx['Date']==date) & (idx["Option_Type"] == "PE") & (idx["Strike"]==strike_pe)]
                                if len(x_ce):
                                    group.loc[idi,'Strike_CE']=x1=x_ce["Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_CE']=x2=x_ce["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_CE']=x3=x_ce["Ticker"].iat[0]
                                    group.loc[idi,'IV_CE']=x_ce["IV"].iat[0]
                                if len(x_ce)==0:
                                    if len(BN_data[(BN_data['Date']==curr_date) & (BN_data['Strike']==a1) & (BN_data['Option_Type']=='CE')]):
                                        group.loc[idi,'Strike_CE']=x1=BN_data[(BN_data['Date']==curr_date) & (BN_data['Strike']==a1) & (BN_data['Option_Type']=='CE')]['Strike'].iat[0]
                                        group.loc[idi,'OPT_Close_CE']=x2=BN_data[(BN_data['Date']==curr_date) & (BN_data['Strike']==a1) & (BN_data['Option_Type']=='CE')]['OPT_Close'].iat[0]
                                        group.loc[idi,'Ticker_CE']=x3=BN_data[(BN_data['Date']==curr_date) & (BN_data['Strike']==a1) & (BN_data['Option_Type']=='CE')]['Ticker'].iat[0]
                                        group.loc[idi,'IV_CE']=np.nan
                                    else:
                                        group.loc[idi,'Strike_CE']=x1#x["Strike"].iat[0]
                                        group.loc[idi,'OPT_Close_CE']=x2#x["OPT_Close"].iat[0]
                                        group.loc[idi,'Ticker_CE']=x3#x["Ticker"].iat[0]
                                        group.loc[idi,'IV_CE']=np.nan#x["IV"].iat[0]



                                if len(x_pe):
                                    group.loc[idi,'Strike_PE']=y1=x_pe["Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_PE']=y2=x_pe["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_PE']=y3=x_pe["Ticker"].iat[0]
                                    group.loc[idi,'IV_PE']=x_pe["IV"].iat[0]
                                if len(x_pe)==0:
                                    if len(BN_data[(BN_data['Date']==curr_date) & (BN_data['Strike']==b1) & (BN_data['Option_Type']=='PE')]):
                                        group.loc[idi,'Strike_PE']=y1=BN_data[(BN_data['Date']==curr_date) & (BN_data['Strike']==a1) & (BN_data['Option_Type']=='PE')]['Strike'].iat[0]
                                        group.loc[idi,'OPT_Close_PE']=y2=BN_data[(BN_data['Date']==curr_date) & (BN_data['Strike']==a1) & (BN_data['Option_Type']=='PE')]['OPT_Close'].iat[0]
                                        group.loc[idi,'Ticker_PE']=y3=BN_data[(BN_data['Date']==curr_date) & (BN_data['Strike']==a1) & (BN_data['Option_Type']=='PE')]['Ticker'].iat[0]
                                        group.loc[idi,'IV_PE']=np.nan
                                    else:
                                        group.loc[idi,'Strike_PE']=y1#"Strike"].iat[0]
                                        group.loc[idi,'OPT_Close_PE']=y2#x["OPT_Close"].iat[0]
                                        group.loc[idi,'Ticker_PE']=y3#x["Ticker"].iat[0]
                                        group.loc[idi,'IV_PE']=np.nan#x["IV"].iat[0]
                                if row['Marker']=='L':
                                    if group.loc[idi,'Strike_PE']>=group.loc[idi,'EQ_Close_CE']:
                                        # print(y1,group.loc[idi,'EQ_Close_CE'],date)
                                        group.loc[idi,'OPT_Close_PE']=-group.loc[idi,'EQ_Close_CE']+group.loc[idi,'Strike_PE']
                                    else:
                                        group.loc[idi,'OPT_Close_PE']=0.05
                                    if group.loc[idi,'Strike_CE']<=group.loc[idi,'EQ_Close_CE']:
                                        group.loc[idi,'OPT_Close_CE']=+group.loc[idi,'EQ_Close_CE']-group.loc[idi,'Strike_CE']
                                    else:
                                        group.loc[idi,'OPT_Close_CE']=0.05
                            if row['Marker']!='F':
                                date=group.loc[idi,'Date']
                                x_ce=og_idx[(og_idx['Date']==date) & (og_idx["Option_Type"] == "CE") & (og_idx["Strike"]==strike_ce)]
                                x_pe=og_idx[(og_idx['Date']==date) & (og_idx["Option_Type"] == "PE") & (og_idx["Strike"]==strike_pe)]
                                if len(x_ce):
                                    group.loc[idi,'Strike_CE']=x1=x_ce["Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_CE']=x2=x_ce["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_CE']=x3=x_ce["Ticker"].iat[0]
                                    group.loc[idi,'IV_CE']=x_ce["IV"].iat[0]
                                if len(x_ce)==0:
                                    group.loc[idi,'Strike_CE']=x1#x["Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_CE']=x2#x["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_CE']=x3#x["Ticker"].iat[0]
                                    group.loc[idi,'IV_CE']=np.nan#x["IV"].iat[0]

                                if len(x_pe):
                                    group.loc[idi,'Strike_PE']=y1=x_pe["Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_PE']=y2=x_pe["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_PE']=y3=x_pe["Ticker"].iat[0]
                                    group.loc[idi,'IV_PE']=x_pe["IV"].iat[0]
                                if len(x_pe)==0:
                                    group.loc[idi,'Strike_PE']=y1#"Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_PE']=y2#x["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_PE']=y3#x["Ticker"].iat[0]
                                    group.loc[idi,'IV_PE']=np.nan#x["IV"].iat[0]
                                if row['Marker']=='L':
                                    if group.loc[idi,'Strike_PE']>=group.loc[idi,'EQ_Close_CE']:
                                        # print(y1,group.loc[idi,'EQ_Close_CE'],date)
                                        group.loc[idi,'OPT_Close_PE']=-group.loc[idi,'EQ_Close_CE']+group.loc[idi,'Strike_PE']
                                    else:
                                        group.loc[idi,'OPT_Close_PE']=0.05
                                    if group.loc[idi,'Strike_CE']<=group.loc[idi,'EQ_Close_CE']:
                                        group.loc[idi,'OPT_Close_CE']=+group.loc[idi,'EQ_Close_CE']-group.loc[idi,'Strike_CE']
                                    else:
                                        group.loc[idi,'OPT_Close_CE']=0.05
                                        
                                        
                            if nnn1==0 and nnn==0:
                                if row['Marker']=='L':

                                    yy_ce=idx2[(idx2['Date']==row['Date']) & (idx2["Option_Type"] == "CE")].reset_index(drop=True)
                                    strike_new_ce=yy_ce["Delta_"+delta+"_Strike"].iat[0]
                                    yy_ce=yy_ce[(yy_ce['Date']==date) & (yy_ce["Option_Type"] == "CE") & (yy_ce["Strike"]==strike_new_ce)]

                                    yy_pe=idx2[(idx2['Date']==row['Date']) & (idx2["Option_Type"] == "PE")].reset_index(drop=True)
                                    strike_new_pe=yy_pe["Delta_"+delta+"_Strike"].iat[0]
                                    yy_pe=yy_pe[(yy_pe['Date']==date) & (yy_pe["Option_Type"] == "PE") & (yy_pe["Strike"]==strike_new_pe)]

                                    group.loc[idi,'Strike_w2_ce']=yy_ce["Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_w2_ce']=x2=yy_ce["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_w2_ce']=yy_ce["Ticker"].iat[0]
                                    group.loc[idi,'IV_w2_ce']=yy_ce["IV"].iat[0]    # xxx
                                    group.loc[idi,'Strike_w2_pe']=yy_pe["Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_w2_pe']=y2=yy_pe["OPT_Close"].iat[0]

                                    group.loc[idi,'Ticker_w2_pe']=yy_pe["Ticker"].iat[0]
                                    group.loc[idi,'IV_w2_pe']=yy_pe["IV"].iat[0] 
                                    temp=1
                                    temp11=1
                                    ce_close_new=group.loc[idi,'OPT_Close_w2_ce']
                                    pe_close_new=group.loc[idi,'OPT_Close_w2_pe']

                                    if temp==0:
                                        temp=1
                                        continue
                                        
                            if row['Marker']=='F':
                                group.loc[idi,'Premium']=group.loc[idi,'OPT_Close_PE']+group.loc[idi,'OPT_Close_CE']
                                group.loc[idi, 'Upper_Breakeven'] = group.loc[idi, 'Strike_CE'] + (group.loc[idi, 'Premium'])
                                group.loc[idi, 'Lower_Breakeven'] = group.loc[idi,'Strike_PE'] - (group.loc[idi, 'Premium'])
                                group.loc[idi, 'Range']=group.loc[idi, 'Upper_Breakeven']-group.loc[idi, 'Lower_Breakeven']
                                
                                group.loc[idi, 'stop_trigger_CE'] = group.loc[idi, 'Strike_CE'] + t*(group.loc[idi, 'Premium'])/100
                                group.loc[idi, 'stop_trigger_PE'] = group.loc[idi,'Strike_PE'] - t*(group.loc[idi, 'Premium'])/100
                                group.loc[idi+1:,'stop_trigger_CE'] = np.nan
                                group.loc[idi+1:,'stop_trigger_PE'] = np.nan
                                group['stop_trigger_CE'].ffill(inplace=True)
                                group['stop_trigger_PE'].ffill(inplace=True)
                                    
                                
                    temp1=0
                    temp_a=0
                    x1=x2=x3=y1=y2=y3=0
                    temp_b=0
                    if nnn!=4:
                        for idi,row in group.iterrows():
                            group.loc[idi,'Premium']=group.loc[idi,'OPT_Close_PE']+group.loc[idi,'OPT_Close_CE']

                            # if nnn1!=0:
                            if nnn==0 and row['Marker']=='L':
                                ce_w2=group.loc[idi,'OPT_Close_w2_ce']
                                pe_w2=group.loc[idi,'OPT_Close_w2_pe']
                            if row['Marker']=='F' and idi!=0:
                                if nnn==0:
                                    start_premium=ce_w2+pe_w2
                                else:
                                    start_premium=group.loc[idi,'Premium']
                            if idi==0:
                                start_premium=group.loc[idi,'Premium']
                                
                            group.loc[idi,'%_Premium_collected']=(start_premium-group.loc[idi,'Premium'])/start_premium
                            
                            
                            if temp_a==1:
                                curr_date=row['Date']
                                x_ce=idx[(idx['Date']==curr_date) & (idx["Option_Type"] == "CE") & (idx["Strike"]==a1)]
                                x_pe=idx[(idx['Date']==curr_date) & (idx["Option_Type"] == "PE") & (idx["Strike"]==b1)]
                                # display(x_ce)
                                # display(x_pe)
                                if len(x_ce):
                                    group.loc[idi,'Strike_CE']=x1=x_ce["Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_CE']=x2=x_ce["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_CE']=x3=x_ce["Ticker"].iat[0]
                                    group.loc[idi,'IV_CE']=x_ce["IV"].iat[0]
                                if len(x_ce)==0:
                                    group.loc[idi,'Strike_CE']=x1#x["Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_CE']=x2#x["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_CE']=x3#x["Ticker"].iat[0]
                                    group.loc[idi,'IV_CE']=np.nan#x["IV"].iat[0]

                                if len(x_pe):
                                    group.loc[idi,'Strike_PE']=y1=x_pe["Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_PE']=y2=x_pe["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_PE']=y3=x_pe["Ticker"].iat[0]
                                    group.loc[idi,'IV_PE']=x_pe["IV"].iat[0]
                                if len(x_pe)==0:
                                    group.loc[idi,'Strike_PE']=y1#"Strike"].iat[0]
                                    group.loc[idi,'OPT_Close_PE']=y2#x["OPT_Close"].iat[0]
                                    group.loc[idi,'Ticker_PE']=y3#x["Ticker"].iat[0]
                                    group.loc[idi,'IV_PE']=np.nan#x["IV"].iat[0]
                                # group.loc[idi,'Premium']=group.loc[idi,'OPT_Close_PE']+group.loc[idi,'OPT_Close_CE']  
                                if x1==0:
                                    xxx
                                if row['Marker']=='L':
                                    if group.loc[idi,'Strike_PE']>=group.loc[idi,'EQ_Close_CE']:
                                        group.loc[idi,'OPT_Close_PE']=-group.loc[idi,'EQ_Close_CE']+group.loc[idi,'Strike_PE']
                                    else:
                                        group.loc[idi,'OPT_Close_PE']=0.05
                                    if group.loc[idi,'Strike_CE']<=group.loc[idi,'EQ_Close_CE']:
                                        group.loc[idi,'OPT_Close_CE']=+group.loc[idi,'EQ_Close_CE']-group.loc[idi,'Strike_CE']
                                    else:
                                        group.loc[idi,'OPT_Close_CE']=0.05
                                group.loc[idi,'Premium']=group.loc[idi,'OPT_Close_PE']+group.loc[idi,'OPT_Close_CE']
                                group.loc[idi, 'Upper_Breakeven'] = group.loc[idi, 'Strike_CE'] + (group.loc[idi, 'Premium'])
                                group.loc[idi, 'Lower_Breakeven'] = group.loc[idi,'Strike_PE'] - (group.loc[idi, 'Premium'])
                                group.loc[idi, 'Range']=group.loc[idi, 'Upper_Breakeven']-group.loc[idi, 'Lower_Breakeven']

                            if (group.loc[idi,'stop_trigger_CE'] <= group.loc[idi,'EQ_Close_CE']) or (group.loc[idi,'stop_trigger_PE'] >= group.loc[idi,'EQ_Close_CE']):
                                if row['Marker']!='L':# and group.loc[idi+1,'Marker']!='L':
                                    temp_b=0
                                    group.loc[idi,'Status1']=1
                                    shift_date= (group.loc[idi,'Date'])
                                    shift_date=pd.to_datetime(shift_date)
                                    ce_new=og_idx[(og_idx['Date']==shift_date) &(og_idx["Strike"] == og_idx["Delta_"+delta+"_Strike"]) & (og_idx["Option_Type"] == "CE")].reset_index(drop=True)
                                    pe_new=og_idx[(og_idx['Date']==shift_date)& (og_idx["Strike"] == og_idx["Delta_"+delta+"_Strike"]) & (og_idx["Option_Type"] == "PE")].reset_index(drop=True)
                                    
                                    group.loc[idi,'new_Strike_CE']=x1=a1=ce_new["Strike"].iat[0]
                                    group.loc[idi,'new_OPT_Close_CE']=x2=a2=ce_new["OPT_Close"].iat[0]
                                    group.loc[idi,'new_Ticker_CE']=x3=a3=ce_new["Ticker"].iat[0]
                                    group.loc[idi,'new_IV_CE']=ce_new["IV"].iat[0]
                                    group.loc[idi,'new_Strike_PE']=y1=b1=pe_new["Strike"].iat[0]
                                    group.loc[idi,'new_OPT_Close_PE']=y2=b2=pe_new["OPT_Close"].iat[0]
                                    group.loc[idi,'new_Ticker_PE']=y3=b3=pe_new["Ticker"].iat[0]
                                    group.loc[idi,'new_IV_PE']=pe_new["IV"].iat[0]
                                    group.loc[idi,'Premium_new']=group.loc[idi,'new_OPT_Close_CE']+group.loc[idi,'new_OPT_Close_PE']
                                    group.loc[idi, 'Upper_Breakeven'] = group.loc[idi, 'new_Strike_CE'] + (group.loc[idi, 'Premium_new'])
                                    group.loc[idi, 'Lower_Breakeven'] = group.loc[idi,'new_Strike_PE'] - (group.loc[idi, 'Premium_new'])
                                    group.loc[idi, 'Range']=group.loc[idi, 'Upper_Breakeven']-group.loc[idi, 'Lower_Breakeven']
                                    group.loc[idi, 'stop_trigger_CE'] = group.loc[idi,'new_Strike_CE'] + t*(group.loc[idi, 'Premium_new'])/100
                                    group.loc[idi, 'stop_trigger_PE'] = group.loc[idi,'new_Strike_PE'] - t*(group.loc[idi, 'Premium_new'])/100
                                    group.loc[idi+1:,'stop_trigger_CE'] = np.nan
                                    group.loc[idi+1:,'stop_trigger_PE'] = np.nan
                                    group['stop_trigger_CE'].ffill(inplace=True)
                                    group['stop_trigger_PE'].ffill(inplace=True)
                                    temp_a=1
                                    continue
                            
                        final=final.append(group,ignore_index=True)

                    else:
                        final=final.append(group,ignore_index=True)
                
                final.to_csv(output_path+'/t3-w3-3-2.csv')
                
                df=pd.read_csv(output_path+'/t3-w3-3-2.csv')
                poe=0.01
                df['lotsize']=1
                df['qty']=1
                plvalue = []
                pl = 0
                transaction_cost = 0
                contract_value=0

                df['Exposure']=df['EQ_Close_CE']*2*df['lotsize']
                df['Margin']=df['Exposure']/leverage


                initial_equity=10000000
                equity=poe*initial_equity
                # df['units'] = (initial_equity/df['Margin']).astype(int)
                # df['qty_final'] = df["qty"]*df['units']

                df.to_csv(output_path+"/unit check1=w3.csv")


                df=pd.read_csv(output_path+"/unit check1=w3.csv")
                # print(6,nnn1)
                temp=0
                       
                for idi, row in df.iterrows():
                        
                        if idi==0:
                            df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                            pl=pl+df.loc[idi,'Points']
                        # if row["Marker"] == "F":
                            df.loc[idi, 'pl_check'] = (pl+initial_equity)
                            df.loc[idi+1:, 'pl_check'] = np.nan
                            
                            (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                            df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                            df.loc[idi+1: , 'units_new'] = np.nan

                            df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                            df.loc[idi+1:, 'qty_final'] = np.nan
                            df['qty_final']=df['qty_final'].ffill()
                            # df.loc[idi, 'plvalue_' + index] = 0
                            # df.loc[idi, 'transaction_cost'] = 0
                            contract_value = df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                            continue

                        if row['Marker2']=='L':# and row['Marker']!='L':
                            df.loc[idi,'CE_Points']=0 #-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"]
                            df.loc[idi,'PE_Points']=0 #-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"]
                            df.loc[idi,'Points']=0 #(-df.loc[idi,  "OPT_Close_PE"]+df.loc[idi-1,  "OPT_Close_PE"])+(-df.loc[idi,  "OPT_Close_CE"]+df.loc[idi-1,  "OPT_Close_CE"])

                        else:
                            if nnn>0:
                                if row["Marker"] == "F":
                                    df.loc[idi,'CE_Points']=df.loc[idi,'PE_Points']=df.loc[idi,'Points']=0
                                    contract_value= df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                                    
                                    
                                # if row["Marker"] == "F":
                                    df.loc[idi, 'pl_check'] = (pl+initial_equity)
                                    df.loc[idi+1:, 'pl_check'] = np.nan

                                    (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                                    df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                                    df.loc[idi+1: , 'units_new'] = np.nan

                                    df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                                    df.loc[idi+1:, 'qty_final'] = np.nan
                                    df['qty_final']=df['qty_final'].ffill()
                                    df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']
                                    pl=pl+df.loc[idi,'Points_final']
                            # df.loc[idi, 'plvalue_' + index] = 0
                            # df.loc[idi, 'transaction_cost'] = 0

                                elif df.loc[idi-1,'Status1']==1:
                                    df.loc[idi,'CE_Points']=df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "new_OPT_Close_CE"]
                                    df.loc[idi,'PE_Points']=df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "new_OPT_Close_PE"]
                                    df.loc[idi,'Points']=df.loc[idi,'CE_Points']+df.loc[idi,'PE_Points']
                                    
                                    contract_value= df.loc[idi, "qty_final"]*(df.loc[idi-1,  "new_OPT_Close_CE"]+df.loc[idi-1,  "new_OPT_Close_PE"])
                                # if row["Marker"] == "F":
                                    df.loc[idi, 'pl_check'] = (pl+ initial_equity)
                                    df.loc[idi+1:, 'pl_check'] = np.nan

                                    (df.loc[idi ,'available_margin']) = (df.loc[idi ,'pl_check'])*leverage

                                    df.loc[idi , 'units_new'] = (((df.loc[idi ,'available_margin']))/df.loc[idi, 'EQ_Close_CE'])
                                    df.loc[idi+1: , 'units_new'] = np.nan

                                    df.loc[idi,  'qty_final'] = df.loc[idi,"qty"] * df.loc[idi, 'units_new']
                                    df.loc[idi+1:, 'qty_final'] = np.nan
                                    df['qty_final']=df['qty_final'].ffill()
                                    df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']
                                    pl=pl+df.loc[idi,'Points_final']
                                    
                                else:
                                    df.loc[idi,'CE_Points']=df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "OPT_Close_CE"]
                                    df.loc[idi,'PE_Points']=df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "OPT_Close_PE"]
                                    df.loc[idi,'Points']=(df.loc[idi,  "OPT_Close_PE"]-df.loc[idi-1,  "OPT_Close_PE"])+(df.loc[idi,  "OPT_Close_CE"]-df.loc[idi-1,  "OPT_Close_CE"])
                                    df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']
                                    pl=pl+df.loc[idi,'Points_final']
                                                                        
                                if row['Marker']=='L' or df.loc[idi,'Status1']==1:
                                    contract_value+= df.loc[idi, "qty_final"]*(df.loc[idi,  "OPT_Close_CE"]+df.loc[idi,  "OPT_Close_PE"])
                                    df.loc[idi,'transaction_cost']=contract_value*0.01
                                    pl=pl-df.loc[idi,'transaction_cost']
                                    df.loc[idi,'Points_final']=df.loc[idi,'Points']*df.loc[idi,'qty_final']-df.loc[idi,'transaction_cost']

                                                                
                                                                    

                df['Points_final_2']=df['Points']*df['qty_final']
                df['Total_points']=df['Points_final'].cumsum()
                # print(7,nnn1)
                df.to_csv(output_path+'/pl_points_'+str(delta)+'.csv')

                df=pd.read_csv(output_path+'/pl_points_'+str(delta)+'.csv')
                temp=0
                df['Total_points']=df['Points_final'].cumsum()
                df['eq_curve']=df['Points_final']
                df.loc[0,'eq_curve']=initial_equity
                df['eq_curve']=df['eq_curve'].cumsum()
                df.to_csv(output_path+'/eq_curve_'+str(delta)+'_'+str(t)+'.csv')
                print((output_path+'/eq_curve_'+str(delta)+'_'+str(t)+'.csv'))
                print('Done')
                # xxx
                
                
                
                